# XY-FZ35: electronic load in custom build housing

## Overview

The **XY-FZ35** is a small and very inexpensive electronic load with a maximum **load power of 35 W** and **load current of 5 A**. Unlike considerably more expensive devices with a wide range of functions, the XY-FZ35 offers "Constant Current (CC)" as its only function. Therefore, it is suitable, for example, to investigate both the performance and the response of various protection circuits of wall mounted power supply outputs, mobile power banks or conditioning batteries.

![Front view of XY-FZ35 build in custom housing](images/XY-FZ35_electronic_load_front.jpeg)  
Front view of XY-FZ35 build in custom housing

The XY-FZ35 is not a ready-to-use device, but is intended for snap-in mounting for installation in a front panel or enclosed housing. My assembly looks like this:

![Inside top view of XY-FZ35 build in custom housing](images/XY-FZ35_electronic_load_inside_view_w_description.png)  
Inside top view of XY-FZ35 build in custom housing

## Technical data

- Snap-in panel device
- Input voltage: 5.0 - 30.0 V DC (reverse polarity protected)
- Load voltage: 1.5 - 25.0 V DC (reverse polarity protected)
- Load current: 0.00 - 5.00 A with 0.01 A resolution
- Load power: 35 W at maximum
- Current regulation: ±(1% + 3 digits)
- Voltage regulation: ±(0.5% + 1 digit)
- Over voltage protection (OVP): default 25.2 V (adjustable)
- Over current protection (OCP): default 5.1O A (adjustable)
- Over power protection (OPP): default 35.5 W (adjustable)
- Over temperature protection (OTP): ~ 80 °C (fixed)

<span style="color:green">**Note:**</span> The display flashes and shows the error code e.g. `OVP` or `OCP`.

- Low voltage protection (LVP): default 1.5 V (adjustable)

<span style="color:green">**Note:**</span> Important for battery discharge tests: Setting the LVP value can prevent battery from deep discharge.


- Operating Temperature: -40 to ~85 °C

The Fan starts automatically when the load power is greater than 10 W or the temperature is greater than 40 °C.

## Literature:

- [Data sheet XY-FZ35](https://m.media-amazon.com/images/I/B1rtWZuqjcS.pdf)
- [XY-FZ35 - Inexpensive Electronic Load](https://community.element14.com/challenges-projects/project14/test-instrumentation/b/blog/posts/xy-fz35---inexpensive-electronic-load)
- [makerspacelt / fz35-cli](https://github.com/makerspacelt/fz35-cli)
- [XY-FZ25/35 Communication Description](https://github.com/ah01/fz35/blob/master/communication.md)
- [XY-FZ25 & XY-FZ35 Electronic Load Control Program](https://github.com/yellobyte/ElectronicLoad-Control-XY-FZ35)
- [FZ35 Adjustable Electronic Load Questions](https://www.eevblog.com/forum/testgear/fz35-adjustabe-electronic-load-questions/)

# Serial interface for controlling the XY-FZ35

Following description was inspired by [https://github.com/ah01/fz35/blob/master/communication.md](https://github.com/ah01/fz35/blob/master/communication.md).

## Electric characteristics

- TTL level communication (5 V level, there is a XL1509-5.0 buck voltage regulator on board)

<span style="color:red">**Warnings:**</span>
- TX and RX pins are connected directly to MCU pins without any protection.
- There is **NO** galvanic isolation between communication interface, power supply or load input.

For easier use, I added a serial to USB converter with **CP2102** chipset.

## Serial connection parameters

| Serial parameter | Setting  |
|------------------|----------|
| Baud Rate        | 9600 bps |
| Data bits        | 8        |
| Stop bits        | 1        |
| Parity           | None     |
| Flow control     | None     |

## Protocol

- serial master-slave communication
- commands are to be sent **without** any line ending (like `CR`, `LF` or both)
- replies ending with `CRLF`

### Commands

Short overview:

| Command     | Reply      | Note                              |
|-------------|------------|-----------------------------------|
| `start`     | S/F        | Start periodic measurement upload |
| `stop`      | S/F        | Stop upload                       |
| `on`        | S/F        | Turn on load function             |
| `off`       | S/F        | Turn off load function            |
| `x.xxA`     | S/F        | Set load current                  |
| `LVP:xx.x`  | S/F        | Set low voltage protection        |
| `OVP:xx.x`  | S/F        | Set over voltage protection       |
| `OCP:x.xx`  | S/F        | Set over current protection       |
| `OPP:xx.xx` | S/F        | Set over power Protection         |
| `OAH:x.xxx` | S/F        | Set maximum capacity              |
| `OHP:xx:xx` | S/F        | Set maximum discharge time        |
| `read`      | parameters | Read product parameter settings   |

Description:
- **LVP:** (Low Voltage Protection) If the voltage drops below a set value then the load will turn itself off. This is important for discharge tests on batteries in order to protect the battery from deep discharge.
- **OAH**: (Maximum Discharge Capacity) When the load is turned on it calculates the accumulated discharge capacity (in Ah) and turns itself off when a set value has been reached. This feature too is for protecting the battery when doing discharge tests.
- **OHP**: (Maximum Discharge Time) When the discharge time reaches a set period of time than the load will turn itself off. Important for discharge tests.
- **OVP**: (Over Voltage Protection) If the voltage is greater then a set value the load will turn itself off.
- **OCP**: (Over Current Protection) If the current is greater then a set value the load will turn itself off.
- **OPP**: (Over Power Protection) If the power it absorbs gets greater then a set value then the load will turn itself off.

<span style="color:green">**Note:**</span> Some alarms (**OPP**, **OAH**, **OHP**) can't be cleared via serial communication. In those cases the On/Off Button on the device itself must be pressed to end the alarm and get the device operational again. Message Boxes will tell you if that's the case.

### S/F Replies (success/fail)

Most commands have a reply just `success` or `fail`.

Fail usualy means wrong format or a value out of range. Especially the format is tricky - you need to send exact same decimal digits as required (including leading and ending zeros) and also **no line ending**!

Examples: `OPP:05.00` will work, but `OPP:5` or `OPP:5.0` or `OPP:05.00<CR><LF>` will not.

<span style="color:green">**Note:**</span> My FZ35 returns in success case `sucess` (sic). But [ah01](https://github.com/ah01/fz35/blob/master/communication.md) stated that there are some implementations of communication library that respond `success` with correct spelling. So maybe there are some more FW versions out there.

### Parameters reply

For `read` command the device will reply with current setting in following format:

```
OVP:xx.x, OCP:x.xx, OPP:xx.xx, LVP:xx.x,OAH:x.xxx,OHP:xx:xx<CR><LF>
```

<span style="color:green">**Note:**</span> Spaces are correct.

### Measurement upload

After `start` command the device will start sending current measurement every 1 second with following format:

```
xx.xxV,x.xA,x.xxxAh,xx:xx<CR><LF>
```

# Test program

Following test program was inspired by [XY-FZ35 - Inexpensive Electronic Load](https://community.element14.com/challenges-projects/project14/test-instrumentation/b/blog/posts/xy-fz35---inexpensive-electronic-load).

In [13]:
import serial
import time

SERIAL_PORT = "/dev/ttyUSB0"
serial = serial.Serial(port=SERIAL_PORT, baudrate=9600, timeout=2)

# Set Protection
OVP = b'OVP:25.2'        # over voltage
OCP = b'OCP:5.00'        # over current
OPP = b'OPP:35.10'       # over power
LVP = b'LVP:01.5'        # low voltage

CMD_WRITE_PAUSE = 0.1    # after writing commands a pause is needed

serial.write(LVP)
time.sleep(CMD_WRITE_PAUSE)
serial.write(OVP)
time.sleep(CMD_WRITE_PAUSE)
serial.write(OCP)
time.sleep(CMD_WRITE_PAUSE)
serial.write(OPP)
time.sleep(CMD_WRITE_PAUSE)
print(serial.read_all(), '\n')      # read setting success

# Vary current and read results
print('Voltage(V), Current(A)')
current = 0.0
for x in range(0, 10, 1):
    current = current + 0.1
    strCurrent = '{:.2f}'.format(current) + 'A'    # must be in exact format
    serial.write(strCurrent.encode('utf-8'))          # byte encoding
    time.sleep(CMD_WRITE_PAUSE)
    serial.read_all()                                 # dummy read
    time.sleep(1)                                  # let things settle
    # check for valid input
    keepLooping = True
    while keepLooping:
        rawResult = (serial.read_all())
        time.sleep(0.1)
        if len(rawResult) > 5:
            if chr(rawResult[5]) == "V":
                result = rawResult.decode('utf-8')                      # decode it - make it a string
                result = result[:12]                                    # strip off end
                result = ''.join(i for i in result if not i.isalpha())  # strip alphabetic chars
                result = ''.join(i for i in result if i.isprintable())  # strip non-printable chars
                if result[0] == '0':
                    result = result[1:]                                 # strip off first zero if present
                print(result)
                keepLooping = False
            serial.flushInput()

b'sucess\r\nsucess\r\n' 

Voltage(V), Current(A)


KeyboardInterrupt: 

In [1]:
import serial
import time

SERIAL_PORT = "/dev/ttyUSB0"
serial = serial.Serial(port=SERIAL_PORT, baudrate=9600, timeout=2)

# Set Protection
OVP = b'OVP:25.2'        # over voltage
OCP = b'OCP:5.00'        # over current
OPP = b'OPP:35.10'       # over power
LVP = b'LVP:01.5'        # low voltage

CMD_WRITE_PAUSE = 0.5    # after writing commands a pause is needed

serial.write(LVP)
time.sleep(CMD_WRITE_PAUSE)
serial.write(OVP)
time.sleep(CMD_WRITE_PAUSE)
serial.write(OCP)
time.sleep(CMD_WRITE_PAUSE)
serial.write(OPP)
time.sleep(CMD_WRITE_PAUSE)
print(serial.read_all(), '\n')      # read setting success
time.sleep(CMD_WRITE_PAUSE)

# read parameters
serial.write('read'.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)
print(serial.read_all(), '\n')

b'sucess\r\nsucess\r\n04.47V,2.40A,1.168Ah,00:29\r\nsucess\r\nsucess\r\n' 

b'04.47V,2.40A,1.168Ah,00:29\r\nOVP:25.2, OCP:5.00, OPP:35.10, LVP:01.5,OAH:0.000,OHP:00:00\r\n' 



In [5]:
current = 0.8
strCurrent = '{:.2f}'.format(current) + 'A'
print(strCurrent)

0.80A


In [6]:
serial.write(strCurrent.encode('utf-8'))

5

In [9]:
print(serial.read_all(), '\n')      # read setting success

b'00.00V,0.00A,0.067Ah,00:01\r\n00.00V,0.00A,0.067Ah,00:01\r\n00.00V,0.00A,0.067Ah,00:01\r\n00.00V,0.00A,0.067Ah,00:01\r\n00.00V,0.00A,0.067Ah,00:01\r\n00.00V,0.00A,0.067Ah,00:01\r\n00.00V,0.00A,0.067Ah,00:01\r\n00.00V,0.00A,0.067Ah,00:01\r\n00.00V,0.00A,0.067Ah,00:01\r\n00.00V,0.00A,0.067Ah,00:01\r\n00.00V,0.00A,0.067Ah,00:01\r\n00.00V,0.00A,0.067Ah,00:01\r\nsucess\r\n' 



In [20]:
# read parameters
serial.write('read'.encode('utf-8'))
print(serial.read_all(), '\n')

b'OVP:25.2, OCP:5.00, OPP:35.10, LVP:01.5,OAH:0.000,OHP:00:00\r\n' 



In [62]:
serial.write('on'.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)
print(serial.read_all(), '\n')      # read setting success

b'04.40V,2.50A,0.008Ah,00:00\r\n04.40V,2.50A,0.009Ah,00:00\r\n04.41V,2.50A,0.009Ah,00:00\r\n04.40V,2.50A,0.010Ah,00:00\r\n04.40V,2.50A,0.011Ah,00:00\r\n04.41V,2.50A,0.011Ah,00:00\r\n04.41V,2.50A,0.012Ah,00:00\r\n04.41V,2.50A,0.013Ah,00:00\r\n04.41V,2.50A,0.013Ah,00:00\r\n04.41V,2.50A,0.014Ah,00:00\r\n04.41V,2.50A,0.015Ah,00:00\r\n04.41V,2.50A,0.016Ah,00:00\r\n04.41V,2.50A,0.016Ah,00:00\r\nsucess\r\n04.40V,2.60A,0.017Ah,00:00\r\n04.40V,2.60A,0.018Ah,00:00\r\n04.39V,2.60A,0.018Ah,00:00\r\n04.39V,2.60A,0.019Ah,00:00\r\n04.40V,2.60A,0.020Ah,00:00\r\n04.40V,2.60A,0.021Ah,00:00\r\n04.39V,2.60A,0.021Ah,00:00\r\nsucess\r\n04.39V,2.70A,0.022Ah,00:00\r\n01.75V,2.70A,0.023Ah,00:00\r\n00.00V,2.70A,0.000Ah,00:00\r\n00.00V,2.70A,0.000Ah,00:00\r\n00.00V,2.70A,0.000Ah,00:00\r\n00.00V,2.70A,0.000Ah,00:00\r\n00.00V,2.70A,0.000Ah,00:00\r\n00.00V,2.70A,0.000Ah,00:00\r\n00.00V,2.70A,0.000Ah,00:00\r\nsucess\r\n00.00V,2.40A,0.000Ah,00:00\r\n00.00V,2.40A,0.000Ah,00:00\r\n00.00V,2.40A,0.000Ah,00:00\r\n00.00V,2

In [8]:
serial.write('start'.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)

In [4]:
serial.flushInput()
time.sleep(1.0)

while True:
    print(serial.read_all())
    time.sleep(1.0)

b'04.47V,2.40A,1.193Ah,00:29\r\n'
b'04.47V,2.40A,1.194Ah,00:29\r\n'
b'04.47V,2.40A,1.194Ah,00:29\r\n'
b'04.47V,2.40A,1.195Ah,00:29\r\n'
b'04.47V,2.40A,1.196Ah,00:29\r\n'
b'04.47V,2.40A,1.196Ah,00:29\r\n'
b'04.47V,2.40A,1.197Ah,00:29\r\n'
b'04.46V,2.40A,1.198Ah,00:29\r\n'
b'04.46V,2.40A,1.198Ah,00:29\r\n'
b'04.47V,2.40A,1.199Ah,00:29\r\n'
b'04.47V,2.40A,1.200Ah,00:30\r\n'
b'04.47V,2.40A,1.200Ah,00:30\r\n'
b'04.47V,2.40A,1.201Ah,00:30\r\n'
b'04.47V,2.40A,1.202Ah,00:30\r\n'
b'04.47V,2.40A,1.202Ah,00:30\r\n'
b'04.47V,2.40A,1.203Ah,00:30\r\n'
b'04.46V,2.40A,1.204Ah,00:30\r\n'
b'04.47V,2.40A,1.204Ah,00:30\r\n'
b'04.47V,2.40A,1.205Ah,00:30\r\n'
b'04.47V,2.40A,1.206Ah,00:30\r\n'
b'04.47V,2.40A,1.206Ah,00:30\r\n'
b'04.47V,2.40A,1.207Ah,00:30\r\n'
b'04.46V,2.40A,1.208Ah,00:30\r\n'
b'04.46V,2.40A,1.208Ah,00:30\r\n'
b'04.47V,2.40A,1.209Ah,00:30\r\n'
b'04.47V,2.40A,1.210Ah,00:30\r\n'
b'04.46V,2.40A,1.210Ah,00:30\r\n'
b'04.46V,2.40A,1.211Ah,00:30\r\n'
b'04.46V,2.40A,1.212Ah,00:30\r\n'
b'04.47V,2.40A

KeyboardInterrupt: 

In [35]:
serial.flushInput()